<a href="https://colab.research.google.com/github/voke-brume/AI-ML/blob/main/AI/ComputerVision/ImageQuantization/NaiveVsFloyd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Image Quantization**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import sys
import cv2
import numpy as np

def coverPalette(N):
    # Return the palette we're using
    return np.linspace(1, 0, 2 ** N)

def reconstructImage(IQ, palette):
    """
    Given a quantized image IQ and their value, return a floating point image
    """
    # opencv is full of these assertions.
    # If you're looking here you may have an error.
    # Check to see what the assertion says
    assert(np.issubdtype(IQ.dtype, np.integer))
    return palette[IQ]

def upscaleNN(I, target_size):
    """
    NN upsample I until it hits a target size but without going over 4096
    """
    h, w = I.shape[:2]
    scale = 1
    while True:
        if min(h * scale, w * scale) >= target_size:
            break
        if max(h * (scale + 1), w * (scale + 1)) > 4096:
            break
        scale += 1
    # usually you should call a library but providing the call here defeats
    # the purpose :)
    shape = (scale, scale) if I.ndim == 2 else (scale, scale, 1)
    return np.kron(I, np.ones(shape))


In [ ]:
def quantize(v, palette):
    """
    Given a scalar v and array of values palette,
    return the index of the closest value
    """
    if np.isscalar(v): return np.abs(palette - v).argmin()
    output = np.zeros( (v.shape[0]), dtype = int )
    for i in range (len(v)): output[i] = np.abs(palette - v[i]).argmin()
    return output

def quantizeNaive(IF, palette):
    """Given a floating-point image return quantized version (Naive)"""
    newImage = np.zeros(IF.shape, dtype = 'uint8')
    for i in range (IF.shape[0]):
      for j in range (IF.shape[1]):
        newImage[i][j] = quantize(IF[i][j], palette)
    return newImage

def quantizeFloyd(IF, palette):
    """
    Given a floating-point image return quantized version (Floyd-Steinberg)
    """
    output = np.zeros(IF.shape, dtype = 'uint8')
    h = IF.shape[0]
    w = IF.shape[1]
    for x in range (h):
      for y in range (w):
        old_value = IF[x][y].copy()
        color_index = quantize(old_value, palette)
        output[x][y] = color_index
        if np.isscalar(color_index): new_value = palette[color_index]
        else: new_value = [palette[i] for i in color_index]
        error = old_value - new_value
        if x + 1 < h: IF[x+1][y] += error*7/16
        if (x - 1 >= 0) and (y + 1 < w): IF[x-1][y+1] += error*3/16
        if y + 1 < w: IF[x][y+1] += error*5/16
        if (y + 1 < w) and (x + 1 < h): IF[x+1][y+1] += error*1/16  
    return output

## **Image Resizing**

In [ ]:
def resizeImage(I, maxDim):
    """Given an image, make sure it's no bigger than maxDim on either side"""
    new_image = I
    h = new_image.shape[0]
    w = new_image.shape[1]
    while h > maxDim or w > maxDim:
        new_image = cv2.resize(new_image, (0, 0), fx=0.50, fy=0.50)
        h = new_image.shape[0]
        w = new_image.shape[1]
    return new_image